<h1> <b>IMPORT LIBRARIES</b></h1>

In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saigo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()


In [4]:
intents = json.loads(open('intents.json').read())

In [5]:
words = []
classes = []
documents = []
ignoreLetters = ['?','!','.',',']

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\saigo/nltk_data'
    - 'C:\\Users\\saigo\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'C:\\Users\\saigo\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'C:\\Users\\saigo\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\saigo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saigo\AppData\Roaming\nltk_data...


True

In [10]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]

In [14]:
words = sorted(set(words))

In [15]:
words

["'s",
 '(',
 ')',
 'AC',
 'AI/ML',
 'AI/Ml',
 'About',
 'Ac',
 'Are',
 'Bye',
 'Cafetaria',
 'Can',
 'Chemical',
 'Civil',
 'College',
 'Computer',
 'Contact',
 'Do',
 'Does',
 'Engineering',
 'Events',
 'Food',
 'Give',
 'Good',
 'Goodbye',
 'HOD',
 'Have',
 'Hello',
 'Hi',
 'Holiday',
 'How',
 'I',
 'IT',
 'Information',
 'Is',
 'Leaving',
 'Library',
 'Mechanical',
 'See',
 'Semester',
 'Sports',
 'Syllabus',
 'Technology',
 'Tell',
 'Thank',
 'Thanks',
 'UNI',
 'What',
 'Whats',
 'When',
 'Where',
 'Which',
 'Who',
 'Will',
 'You',
 'a',
 'about',
 'active',
 'activity',
 'address',
 'admision',
 'admission',
 'against',
 'allotment',
 'am',
 'an',
 'and',
 'antiragging',
 'any',
 'anyone',
 'are',
 'as',
 'asshole',
 'at',
 'attend',
 'automobile',
 'available',
 'average',
 'be',
 'between',
 'big',
 'bitch',
 'book',
 'boy',
 'branch',
 'bring',
 'building',
 'by',
 'bye',
 'cafetaria',
 'call',
 'called',
 'campus',
 'can',
 'canteen',
 'capacity',
 'case',
 'casuals',
 'ce',


In [12]:
classes = sorted(set(classes))

In [16]:
classes

['admission',
 'canteen',
 'college intake',
 'committee',
 'computerhod',
 'course',
 'creator',
 'document',
 'event',
 'extchod',
 'facilities',
 'fees',
 'floors',
 'goodbye',
 'greeting',
 'hod',
 'hostel',
 'hours',
 'infrastructure',
 'ithod',
 'library',
 'location',
 'menu',
 'name',
 'number',
 'placement',
 'principal',
 'ragging',
 'random',
 'salutaion',
 'scholarship',
 'sem',
 'sports',
 'swear',
 'syllabus',
 'task',
 'uniform',
 'vacation']

In [13]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

<h1><b>Prepare Model</b></h1>

In [17]:
training = []
outputEmpty = [0] * len(classes)

In [18]:
for document in documents:
    bag = []
    wordPatterens = document[0]
    wordPatterens = [lemmatizer.lemmatize(word.lower()) for word in wordPatterens]
    for word in words:
        bag.append(1) if word in wordPatterens else bag.append(0)
    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [19]:
random.shuffle(training)

In [20]:
training = np.array(training)

In [21]:
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
trainX = training[:, :len(words)]

In [23]:
trainY = training[:, len(words):]

In [24]:
trainY

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

<h1><B>Create Model</B></h1>

In [25]:
model = tf.keras.Sequential()

In [26]:
model.add(tf.keras.layers.Dense(128, input_shape = (len(trainX[0]),), activation = 'relu'))

C:\Users\saigo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.add(tf.keras.layers.Dropout(0.5))

In [28]:
model.add(tf.keras.layers.Dense(64, activation = 'relu'))

In [29]:
model.add(tf.keras.layers.Dropout(0.5))

In [30]:
model.add(tf.keras.layers.Dense(len(trainY[0]), activation ='softmax'))

In [31]:
sgd = tf.keras.optimizers.SGD(learning_rate = 0.01 , momentum=0.9, nesterov = True )

In [32]:
model.compile(loss ='categorical_crossentropy', optimizer= sgd , metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          39,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 38)                  │           2,470 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,022 (195.40 KB)

 Trainable params: 50,022 (195.40 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.fit(trainX,trainY,epochs = 200,batch_size=5, verbose= 1)
hist = model.fit(np.array(trainX),np.array(trainY),epochs = 200 ,batch_size = 5 , verbose = 1)
model.save('chat_model.h5',hist)
print('Done')

Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9785 - loss: 0.0815
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9445 - loss: 0.1329
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9720 - loss: 0.1022
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9746 - loss: 0.0612
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9845 - loss: 0.0584
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9836 - loss: 0.0662
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9592 - loss: 0.1310
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9644 - loss: 0.1147
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9751 - loss: 0.0763
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9737 - loss: 0.1035
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9436 - loss: 0.2663
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy:

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9805 - loss: 0.1054
Epoch 70/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9675 - loss: 0.0999
Epoch 71/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9514 - loss: 0.1138
Epoch 72/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9701 - loss: 0.1128
Epoch 73/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9462 - loss: 0.1402
Epoch 74/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9723 - loss: 0.1128
Epoch 75/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9775 - loss: 0.0761
Epoch 76/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9631 - loss: 0.1334
Epoch 77/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9907 - loss: 0.0489
Epoch 78/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9586 - loss: 0.1231
Epoch 79/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9554 - loss: 0.1270
Epoch 80/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9678 - loss: 0.1407
Epoch 138/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9650 - loss: 0.0758
Epoch 139/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9677 - loss: 0.1041
Epoch 140/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9750 - loss: 0.0777
Epoch 141/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9640 - loss: 0.1230
Epoch 142/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9638 - loss: 0.0830
Epoch 143/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9667 - loss: 0.2076
Epoch 144/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9652 - loss: 0.1042
Epoch 145/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9660 - loss: 0.1413
Epoch 146/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9493 - loss: 0.1373
Epoch 147/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9826 - loss: 0.0722
Epoch 148/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - ac

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9668 - loss: 0.0913
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9703 - loss: 0.1133
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9707 - loss: 0.1148
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9643 - loss: 0.1018
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9437 - loss: 0.1625
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9606 - loss: 0.1184
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9336 - loss: 0.2428
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9713 - loss: 0.1219
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9568 - loss: 0.1870
Epoch 13/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9598 - loss: 0.0938
Epoch 14/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9427 - loss: 0.2165
Epoch 15/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9750 -

81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9750 - loss: 0.0891
Epoch 73/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9620 - loss: 0.1069
Epoch 74/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9529 - loss: 0.1607
Epoch 75/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9739 - loss: 0.0669
Epoch 76/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9641 - loss: 0.1233
Epoch 77/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9607 - loss: 0.1359
Epoch 78/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9613 - loss: 0.0923
Epoch 79/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9674 - loss: 0.1023
Epoch 80/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9610 - loss: 0.0809
Epoch 81/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9638 - loss: 0.0982
Epoch 82/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9432 - loss: 0.2129
Epoch 83/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9711 - loss: 0.0631
Epoch 141/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9627 - loss: 0.1784
Epoch 142/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9373 - loss: 0.1777
Epoch 143/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9752 - loss: 0.1430
Epoch 144/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9568 - loss: 0.2922
Epoch 145/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9811 - loss: 0.0763
Epoch 146/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9625 - loss: 0.1303
Epoch 147/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9484 - loss: 0.1648
Epoch 148/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9679 - loss: 0.0937
Epoch 149/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9668 - loss: 0.0996
Epoch 150/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9452 - loss: 0.1453
Epoch 151/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - ac

Done
